In [111]:
import eng_module.columns as columns
import eng_module.sections_db as db
import eng_module.load_factors as lf

import math

combos = {
    "LC1": {"D": 1.4},
    "LC2": {"D": 1.2, "L": 1.6},
    "LC3": {"D": 1.2, "L": 1.0, "W": 1.0}
}

In [112]:
def calculate_moi_min(p_factored: float, h: float, E: float, k: float, phi: float) -> float:
    """
    Returns the min second moment of area required for an axial member
    to resist p_factored based on the theoretical euler buckling load
    P factored is in units of pounds, H is in units of feet
    """
    return (p_factored/1000 * (h*12)**2) / (k * E * math.pi**2) / phi

Load HSS Table

In [113]:
hss_df = db.aisc_hss_sections(si_units=False)

sq_hss_df = hss_df.loc[hss_df['Ix'] == hss_df["Iy"]].reset_index(drop=True)
# sq_hss_df

# Inputs

In [114]:
h_bay = 10 #ft
w_bay = 15 #ft
P_wind = 10000 # lb

w_dead = 500 #plf
w_live = 1000 # plf


# Loading Configuration
Calculated

In [120]:

angle = math.atan(h_bay / w_bay)
l_brace = math.sqrt(h_bay**2 + w_bay**2)

brace_load = {
    "W_load": P_wind / math.sin(angle)
}
column_load = {
    "W_load": P_wind * h_bay / w_bay,
    "D_load": w_dead * w_bay / 2,
    "L_load": w_live * w_bay / 2,
}
beam_load = {
    "D_load": w_dead * w_bay**2/8,
    "L_load": w_live * w_bay**2/8,
}

Pf_col = lf.max_factored_load(column_load, combos)
print(f"{Pf_col=}")
Pf_brace = lf.max_factored_load(brace_load, combos)
Mf_beam = lf.max_factored_load(beam_load, combos)
print(f"{Pf_brace=}")
print(f"{l_brace=}")
print(f"{Mf_beam=}")

Pf_col=18666.666666666668
Pf_brace=18027.75637731995
l_brace=18.027756377319946
Mf_beam=61875.0


### Calculate required MoI

In [116]:
I_min_column = calculate_moi_min(Pf_col,h_bay,29000,1,0.85)
I_min_brace = calculate_moi_min(Pf_brace,l_brace,29000,1,0.85)

### Determine Appropriate Section from Database

In [117]:
sub_df_col = db.section_filter(sq_hss_df, "ge", Ix=I_min_column)
col_section = db.sort_by_weight(sub_df_col).head(1)["Section"].squeeze()

sub_df_brace = db.section_filter(sq_hss_df, "ge", Ix=I_min_brace)
brace_section = db.sort_by_weight(sub_df_brace).head(1)["Section"].squeeze()

print(f"{I_min_brace=} | {I_min_column=}")
print(f"{col_section=} | {brace_section=}")

I_min_brace=3.467934327045745 | I_min_column=1.104873596878718
col_section='HSS3X3X1/8' | brace_section='HSS4X4X1/8'


### Calculate Member Weight

In [118]:
col_li_w = sq_hss_df.loc[sq_hss_df['Section'] == col_section]['W'].squeeze()
brace_li_w =  sq_hss_df.loc[sq_hss_df['Section'] == brace_section]['W'].squeeze()

col_weight = col_li_w * h_bay * 2
brace_weight = brace_li_w * l_brace
beam_weight = w_bay **2

print(f"{col_weight=} lbs | {brace_weight=} lbs | {beam_weight=} lbs")


col_weight=95.0 lbs | brace_weight=116.45930619748685 lbs | beam_weight=225 lbs
